# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [82]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode
from collections import defaultdict, Counter

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)
def convert_word(word):
    return "$" + word

def bigram(corpus):
    bigram_counts = defaultdict(Counter)
    
    for word in corpus:
        word = convert_word(word)
        
        # generate a list of bigrams
        bigram_list = zip(word[:-1], word[1:])
        
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
    return bigram_counts
def build_dictionary( dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    # keep only unique letters in word. remove other letters
    # full_dictionary = [''.join(list(set(word))) for word in full_dictionary]
    text_file.close()
    return full_dictionary
def build_dictionary_bigram(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    # keep only unique letters in word. remove other letters
    full_dictionary = [''.join(list(set(word))) for word in full_dictionary]
    text_file.close()
    return full_dictionary
training_set=build_dictionary_bigram("words_250000_train.txt")
bigram_counts = bigram(training_set)
# Calculate bigram probability
def bigram_prob(key, char, bigram_counts):
    prev_word_counts = bigram_counts[key]
    total_counts = float(sum(prev_word_counts.values()))
    # print("key:",key,"char:",char,"total_counts:",total_counts)
    return prev_word_counts[char] / float(sum(prev_word_counts.values())+26*26)

def trigram_prob(key, key2, char, bigram_counts):
    prev_word_counts = bigram_counts[key][key2]
    total_counts = float(sum(prev_word_counts.values()))
    # print("key:",key,"char:",char,"total_counts:",total_counts)
    return prev_word_counts[char] / float(sum(prev_word_counts.values())+26*26*26)
    

print(bigram_counts['a'])

Counter({'c': 38707, 'l': 20726, 'v': 10723, 'g': 10380, 'b': 10268, 'f': 8440, 'd': 8154, 'i': 6712, 'y': 5148, 'h': 4593, 'e': 4097, 'u': 937, 'q': 829, 'n': 590, 't': 333, 'x': 172, 'w': 145, 'm': 43, 'j': 29, 'p': 11, 'k': 6, 's': 4, 'o': 1})


In [83]:
import nltk
# create letter wise ngrams
def trigram(corpus):
    trigram_counts = defaultdict(lambda: defaultdict(Counter))
    
    for word in corpus:
        word = convert_word(word)
        
        # generate a list of trigrams
        trigram_list = zip(word[:-2], word[1:-1], word[2:])
        
        # iterate over trigrams
        for trigram in trigram_list:
            first, second, third = trigram
            trigram_counts[first][second][third] += 1
    return trigram_counts

# print(training_set[100:104])
# print(trigram(training_set[100:104]))
trigram_counts = trigram(training_set)


In [84]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
    def _guess(self, word,letter): # word input example: "_ p p _ e "
        clean_word = word.replace("_",".")
        len_word = len(clean_word)
        current_dictionary = self.current_dictionary
        # print("current_dictionary:",current_dictionary)
        new_dictionary = []
        for dict_word in current_dictionary:
            if len(dict_word) != len_word:
                continue
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        self.current_dictionary = new_dictionary
        full_dict_string = "".join(new_dictionary)
        if len(full_dict_string) == 0:
            full_dict_string="".join(self.full_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        # print("current_dictionary:",c)
        # print("string:",full_dict_string)
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        return c[letter]/len(full_dict_string)
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                return instance_count/float(len(full_dict_string))
            
        sorted_letter_count = self.full_dictionary_common_letter_sorted
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                return instance_count/float(len(full_dict_string))

    def bigram_guesser(self,mask, guessed, bigram_counts=bigram_counts): # add extra arguments if needed 
        available = list(set(string.ascii_lowercase) - set(guessed))
        bigram_probs = []
        for char in available:
            char_prob = 0
            for index in range(len(mask)):
                # print("index:",index,"char:",char,"mask:",mask,"mask[index]:",mask[index])
                # print("char:",char,"index:",index,"mask[i]:",mask[index])
                if index == 0 and mask[index] == '_':
                    # char_prob +=  bigram_prob('$', char, bigram_counts)
                    char_prob +=  self._guess(mask,char)

                elif mask[index] == '_':
                    if not mask[index - 1] == '_':
                        char_prob +=  bigram_prob(mask[index - 1], char, bigram_counts)
                    else:
                        char_prob +=  self._guess(mask,char)
                else:
                    continue
            bigram_probs.append(char_prob)
        return available[bigram_probs.index(max(bigram_probs))]

    def trigram_guesser(self,mask, guessed, trigram_counts=trigram_counts):
        available = list(set(string.ascii_lowercase) - set(guessed))
        trigram_probs = []
        for char in available:
            char_prob = 0
            for index in range(len(mask)):
                if index == 0 and mask[index] == '_':
                    char_prob +=  self._guess(mask,char)
                elif index == 1 and mask[index] == '_':
                    if not mask[index - 1] == '_':
                        char_prob +=  bigram_prob(mask[index - 1],char,bigram_counts)
                    else:
                        char_prob +=  self._guess(mask,char)
                elif mask[index] == '_':
                    if not mask[index - 1] == '_':
                        if not mask[index - 2] == '_':
                            char_prob +=  trigram_prob(mask[index - 2], mask[index - 1], char, trigram_counts)
                        else:
                            char_prob +=  bigram_prob(mask[index - 1],char, bigram_counts)
                    else:
                        char_prob +=  self._guess(mask,char)
                else:
                    continue
            trigram_probs.append(char_prob)
        return available[trigram_probs.index(max(trigram_probs))]
    
    def guess(self, word):
        word = word[::2].replace("_","_")
        # print("clean_word:",clean_word)
        # return self.bigram_guesser(word, self.guessed_letters, bigram_counts=bigram_counts)
        return self.trigram_guesser(word, self.guessed_letters, trigram_counts=trigram_counts)


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def offline(self):
        tries_remains=1
        self.current_dictionary = self.full_dictionary
        while tries_remains>0:
                # get guessed letter from user code
            word="_ _ _ _ _ "
            guess_letter = self.guess(word)
                
            # append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            print("Guessing letter: {0}".format(guess_letter))
            tries_remains -=1

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                print("Word: {0}".format(word))
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                    print("-")
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    


# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [85]:
api = HangmanAPI(access_token="1351d29da90b3d587ad0f8b1720f69", timeout=2000)
api.offline()

Guessing letter: a


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [99]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 1bf2c4fcedd3. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Word: _ _ _ _ _ _ _ _ 
Guessing letter: e
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tries_remains': 6, 'word': 'e _ _ _ _ _ _ e '}
Word: e _ _ _ _ _ _ e 
Guessing letter: u
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tries_remains': 6, 'word': 'e _ u _ _ _ _ e '}
Word: e _ u _ _ _ _ e 
Guessing letter: i
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ u _ _ _ _ e '}
Word: e _ u _ _ _ _ e 
Guessing letter: t
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ u _ _ _ t e '}
Word: e _ u _ _ _ t e 
Guessing letter: a
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ u _ _ a t e '}
Word: e _ u _ _ a t e 
Guessing letter: c
-
Sever response: {'game_id': '1bf2c4fcedd3', 'status': 'ongoing', 'tr

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [9]:
for i in range(100):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game


KeyboardInterrupt: 

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [16]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
print('run %d recorded games out of an allotted 100,000. recorded success rate so far = %.3f' % (total_recorded_runs, total_recorded_successes))
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

run 4 practice games out of an allotted 100,000. practice success rate so far = 0.250
run 0 recorded games out of an allotted 100,000. recorded success rate so far = 0.000


ZeroDivisionError: division by zero